In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2022-11-24 23:23:23.727123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 23:23:23.840266: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 23:23:23.840283: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-24 23:23:23.858924: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-24 23:23:24.303369: W tensorflow/stream_executor/platform/de

SystemError: GPU device not found

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# packages and modules from ml article linked here: https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

# ignore some of above, trying a different model!
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

# for classifier 
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# other
from utils import type_to_int, type_to_int_dict_inv

In [85]:
url='https://drive.google.com/file/d/1VzeIrzna8SDkJCnIrCIxn5LXnpx3WcrA/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
poke = pd.read_csv(url) #import Pokemon data

url2='https://drive.google.com/file/d/1WR9i10kZ6bkI_7WV4fWmOXXNlaKgnBZ0/view?usp=share_link'
url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]
training_data = pd.read_csv(url2) #import training data

poke = poke[poke.is_legendary == 0] #clean data for no legendaries
poke.reset_index(drop = True,inplace = True)
poke = poke.drop([131, 195, 196, 229, 348, 717]) #clean out problem children (Ditto, etc)
poke.reset_index(drop = True,inplace = True)

In [86]:
#zeroth row data point
pokemon_name = training_data.iloc[0][1] #pokemon 1 of our team is index 1 for column
pokemon_name_last = training_data.iloc[0][11] #last pokemon of opposing team is index 11 for column

#get desired stats
temp = poke.loc[poke["name"]==pokemon_name].squeeze()
#temp["name"],temp["hp"],[temp["type1"],temp["type2"]],temp["attack"],temp["defense"],temp["sp_attack"],temp["sp_defense"],temp["speed"]

In [87]:
training_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,0,Trevenant,Grimer,Sawk,Zorua,Tangela,Meowstic,Vulpix,Forretress,Wishiwashi,...,Hydreigon,Hypno,Delphox,Bisharp,Blastoise,Empoleon,Jolteon,Oranguru,Slowking,Kangaskhan
1,1,Arcanine,Tyrantrum,Venusaur,Budew,Kangaskhan,Braixen,Lickilicky,Victreebel,Espeon,...,Sneasel,Weavile,Greninja,Scizor,Slaking,Minior,Goodra,Ribombee,Lucario,Aerodactyl
2,2,Sawk,Butterfree,Poliwhirl,Tynamo,Seaking,Floette,Quagsire,Litten,Cloyster,...,Silvally,Seviper,Alakazam,Swalot,Oricorio,Delphox,Nidoqueen,Salazzle,Steelix,Typhlosion
3,3,Butterfree,Wurmple,Kangaskhan,Vibrava,Corsola,Toucannon,Hitmonlee,Ursaring,Karrablast,...,Greninja,Alakazam,Manectric,Jellicent,Absol,Mienshao,Porygon-Z,Arcanine,Aegislash,Chimecho
4,4,Elgyem,Mamoswine,Ambipom,Greninja,Caterpie,Toucannon,Magcargo,Stunky,Torracat,...,Purrloin,Liepard,Pansage,Simisage,Pansear,Simisear,Servine,Phione,Froslass,Gallade


In [84]:
# training_data.iloc[3]

In [88]:
X = training_data.iloc[:,1:12]
Y = training_data.iloc[:,12:-1 ]

In [99]:
pokemon_features_df = pd.read_csv('pokemon_features.csv', index_col='name')
type_cols = pokemon_features_df.columns[:18]
stat_cols = pokemon_features_df.columns[18:]
typesX = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in X.iterrows()], axis=0)
statsX = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in X.iterrows()], axis=0)
typesY = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in Y[['11']].iterrows()], axis=0)
statsY = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in Y[['11']].iterrows()], axis=0)

print(typesX[0], statsX[0], '\n')
print(typesY[0], statsY[0])

[0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0] [474 166 167 325 105 130 465 210 150 330 145  80 435 160 180 466 187 155
 299 115 103 465 130 215 620 170 180 535 220 165 355 131 122] 

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0] [600 325 160]


In [14]:
# don't need this right now, it's for a single output binary classifier (from previous reference article) TO DELETE
# X_train, X_test, y_train, y_test = train_test_split(featureX.transpose(), featureY.transpose(), test_size=0.30, random_state=40)
# print(X_train.shape); print(X_test.shape)

For our type output, this could be a binary classifer with classes 1 (is the type) and 0 (is not the type) with 18 outputs. 

For the stats predictor, we cannot use a classifier unless we had all possible classes (every integer option: not feasible), instead we should use a regression model?)

Idea: two different models, one for classifying on type and one for regression

Idea2: use entirely a regression model and pick the type 1/2 that are the closest to 1

In [97]:
#regr = MultiOutputRegressor(Ridge(random_state=123)).fit(featureX, featureY)

In [96]:
#regr.predict(featureX[[0]])

This worked kinda but also not! Stats seems like reasonable numbers for stats, but type doesn't make much sense. Try: two models, one for type (classifier multioutput) and one for stats (regressor multioutput) and then use both of these results and combine them for our total output :)

In [90]:
regr = MultiOutputRegressor(Ridge(random_state = 123)).fit(statsX, statsY)

In [91]:
statsPredict=regr.predict(statsX)

In [35]:
statsPredict

array([[643.88147839, 250.54993431, 211.06853559],
       [645.92533108, 259.55498623, 201.49643557],
       [626.93805044, 248.83794125, 196.62193455],
       ...,
       [599.04693393, 234.27753667, 193.15007847],
       [642.12454779, 256.59643234, 196.54247578],
       [603.08779872, 219.23035738, 205.27514673]])

In [92]:
clf = MultiOutputClassifier(LogisticRegression()).fit(typesX, typesY)

In [93]:
typesY[-1:]

array([[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [94]:
clf.predict(typesX[-1:])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

This binary multiclass output does not return all outputs as having a type. We really need it to return 1 or 2 types for our output pokemon (this is a BIG part of choosing pokemon to battle). New idea: make each type a class with an int 0-17, or 1-18, and maybe another class as a placeholder for a type when a pokemon has only 1 type (or simply have [18 18]. Then, use a multiclass classifier with an output of 2 (representing types)

Need to make data changes :'(

In [95]:
pred = clf.predict(typesX)
np.where(np.sum(pred,axis=1) != 0)[0]

array([   0,    3,    8,   15,   16,   19,   22,   23,   28,   29,   30,
         35,   42,   44,   46,   48,   49,   53,   56,   57,   62,   63,
         64,   66,   72,   75,   76,   79,   94,   96,   98,  104,  106,
        108,  109,  113,  125,  127,  130,  133,  135,  141,  144,  147,
        152,  159,  160,  163,  169,  176,  178,  179,  182,  185,  186,
        187,  196,  197,  198,  200,  201,  202,  203,  204,  208,  209,
        210,  212,  214,  215,  219,  230,  233,  237,  238,  242,  246,
        248,  251,  252,  255,  257,  258,  261,  262,  264,  265,  267,
        268,  269,  271,  273,  278,  279,  280,  281,  282,  285,  286,
        289,  292,  293,  296,  302,  307,  310,  311,  314,  320,  321,
        327,  330,  335,  336,  337,  338,  341,  346,  361,  364,  366,
        367,  368,  370,  374,  375,  376,  377,  379,  384,  386,  387,
        389,  390,  395,  396,  397,  398,  399,  400,  403,  404,  409,
        411,  412,  415,  416,  425,  427,  429,  4

In [32]:
pred[3]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

MAYBE types doooooo only matter in 1/3 of our battles ?? (:

If type isn't  all zero, then condition on type and search for nearest neighbor of that type?
weight based on the team's deficiency
